In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

ModuleNotFoundError: No module named 'google.colab'

In [3]:
!pip install librosa soundfile numpy scikit-learn pyaudio

     ---------------------------------------- 0.0/2.7 MB ? eta -:--:--
     ---------------------------------------- 2.7/2.7 MB 20.5 MB/s eta 0:00:00
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'error'


  error: subprocess-exited-with-error
  
  × Getting requirements to build wheel did not run successfully.
  │ exit code: 1
  ╰─> [24 lines of output]
      Traceback (most recent call last):
        File "C:\Users\sonig\AppData\Local\Programs\Python\Python313\Lib\site-packages\pip\_vendor\pyproject_hooks\_in_process\_in_process.py", line 353, in <module>
          main()
          ~~~~^^
        File "C:\Users\sonig\AppData\Local\Programs\Python\Python313\Lib\site-packages\pip\_vendor\pyproject_hooks\_in_process\_in_process.py", line 335, in main
          json_out['return_val'] = hook(**hook_input['kwargs'])
                                   ~~~~^^^^^^^^^^^^^^^^^^^^^^^^
        File "C:\Users\sonig\AppData\Local\Programs\Python\Python313\Lib\site-packages\pip\_vendor\pyproject_hooks\_in_process\_in_process.py", line 118, in get_requires_for_build_wheel
          return hook(config_settings)
        File "C:\Users\sonig\AppData\Local\Temp\pip-build-env-9uocun5u\overlay\Lib\site-packa

In [8]:
import librosa
import soundfile
import os, glob, pickle
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score

ModuleNotFoundError: No module named 'librosa'

In [ ]:
# DataFlair - Extract features (mfcc, chroma, mel) from a sound file
def extract_feature(file_name, mfcc, chroma, mel):
    with soundfile.SoundFile(file_name) as sound_file:
        X = sound_file.read(dtype="float32")
        sample_rate = sound_file.samplerate
        if chroma:
            stft = np.abs(librosa.stft(X))
        result = np.array([])
        if mfcc:
            mfccs = np.mean(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=40).T, axis=0)
            result = np.hstack((result, mfccs))
        if chroma:
            chroma = np.mean(librosa.feature.chroma_stft(S=stft, sr=sample_rate).T, axis=0)
            result = np.hstack((result, chroma))
        if mel:
            mel = np.mean(librosa.feature.melspectrogram(y=X, sr=sample_rate).T, axis=0)
            result = np.hstack((result, mel))
        return result


In [ ]:
#DataFlair - Emotions in the RAVDESS dataset
emotions={
  '01':'neutral',
  '02':'calm',
  '03':'happy',
  '04':'sad',
  '05':'angry',
  '06':'fearful',
  '07':'disgust',
  '08':'surprised'
}

#DataFlair - Emotions to observe
observed_emotions=['calm', 'happy', 'fearful', 'disgust']

In [ ]:
#DataFlair - Load the data and extract features for each sound file
def load_data(test_size=0.2):
    x,y=[],[]
    for file in glob.glob("/content/drive/MyDrive/speech-emotion-recognition-ravdess-data/Actor_*/*.wav"):
        file_name=os.path.basename(file)
        emotion=emotions[file_name.split("-")[2]]
        if emotion not in observed_emotions:
            continue
        feature=extract_feature(file, mfcc=True, chroma=True, mel=True)
        x.append(feature)
        y.append(emotion)
    return train_test_split(np.array(x), y, test_size=test_size, random_state=9)

In [ ]:
#DataFlair - Split the dataset
x_train,x_test,y_train,y_test=load_data(test_size=0.25)

In [ ]:
#DataFlair - Get the shape of the training and testing datasets
print((x_train.shape[0], x_test.shape[0]))

In [ ]:
#DataFlair - Get the number of features extracted
print(f'Features extracted: {x_train.shape[1]}')

In [ ]:
#DataFlair - Initialize the Multi Layer Perceptron Classifier
model=MLPClassifier(alpha=0.01, batch_size=256, epsilon=1e-08, hidden_layer_sizes=(300,), learning_rate='adaptive', max_iter=500)

In [ ]:
#DataFlair - Train the model
model.fit(x_train,y_train)

In [ ]:
#DataFlair - Predict for the test set
y_pred=model.predict(x_test)

In [ ]:
#DataFlair - Calculate the accuracy of our model
accuracy=accuracy_score(y_true=y_test, y_pred=y_pred)

#DataFlair - Print the accuracy
print("Accuracy: {:.2f}%".format(accuracy*100))

In [ ]:
from sklearn.model_selection import GridSearchCV

# Define a parameter grid to search over
param_grid = {
    'hidden_layer_sizes': [(100,), (200,), (300,)],
    'alpha': [0.0001, 0.001, 0.01],
}

# Create the GridSearchCV object
grid_search = GridSearchCV(

    MLPClassifier(learning_rate='adaptive', max_iter=500),
    param_grid,
    cv=5,  # 5-fold cross-validation
)

# Fit the model with the best hyperparameters
grid_search.fit(x_train, y_train)

# Get the best parameters
best_params = grid_search.best_params_

# Train a model with the best hyperparameters
best_model = MLPClassifier(learning_rate='adaptive', max_iter=500, **best_params)
best_model.fit(x_train, y_train)
y_pred = best_model.predict(x_test)
accuracy = accuracy_score(y_true=y_test, y_pred=y_pred)
print("Best Model Accuracy: {:.2f}%".format(accuracy * 100))


In [ ]:
!pip install audiomentations


In [ ]:
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score

# Hyperparameter tuning with GridSearchCV
param_grid = {
    'hidden_layer_sizes': [(100,), (200,), (300,), (400,)],
    'alpha': [0.0001, 0.001, 0.01, 0.1],
    'learning_rate': ['constant', 'adaptive'],
}

grid_search = GridSearchCV(
    MLPClassifier(max_iter=1000),  # Adjust max_iter as needed
    param_grid,
    cv=5,
)

grid_search.fit(x_train, y_train)
best_params = grid_search.best_params_

best_model = MLPClassifier(max_iter=1000, **best_params)
best_model.fit(x_train, y_train)
y_pred = best_model.predict(x_test)
accuracy = accuracy_score(y_true=y_test, y_pred=y_pred)
print("Best Model Accuracy: {:.2f}%".format(accuracy * 100))

In [ ]:
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score
import librosa

# Hyperparameter tuning with GridSearchCV
param_grid = {
    'hidden_layer_sizes': [(100,), (200,), (300,), (400,)],
    'alpha': [0.0001, 0.001, 0.01, 0.1],
    'learning_rate': ['constant', 'adaptive'],
}

grid_search = GridSearchCV(
    MLPClassifier(max_iter=1000),  # Adjust max_iter as needed
    param_grid,
    cv=5,
)

grid_search.fit(x_train, y_train)
best_params = grid_search.best_params_

best_model = MLPClassifier(max_iter=1000, **best_params)
best_model.fit(x_train, y_train)
y_pred = best_model.predict(x_test)
accuracy = accuracy_score(y_true=y_test, y_pred=y_pred)
print("Best Model Accuracy: {:.2f}%".format(accuracy * 100))


In [ ]:
import joblib,pickle

# After training your model, save it to a file
joblib.dump(model, '/content/drive/MyDrive/model.pkl')


In [ ]:
model = joblib.load('/content/drive/MyDrive/model.pkl')

In [ ]:
!pip install audiomentations ipywidgets matplotlib
import ipywidgets as widgets
from IPython.display import display, Audio
from sklearn.neural_network import MLPClassifier
import librosa
import soundfile as sf
import numpy as np
import io
import joblib
from audiomentations import Compose, AddGaussianNoise, TimeStretch, PitchShift
import matplotlib.pyplot as plt
import librosa.display

# Data augmentation using audiomentations
augment = Compose([
    AddGaussianNoise(min_amplitude=0.001, max_amplitude=0.015, p=0.5),
    TimeStretch(min_rate=0.8, max_rate=1.2, p=0.5),
    PitchShift(min_semitones=-4, max_semitones=4, p=0.5),
])

# Extract features function (with mel, mfcc, and chroma)
def extract_features(X, sample_rate, mfcc, chroma, mel):
    if chroma:
        stft = np.abs(librosa.stft(X))
    result = np.array([])
    if mfcc:
        mfccs = np.mean(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=40).T, axis=0)
        result = np.hstack((result, mfccs))
    if chroma:
        chroma = np.mean(librosa.feature.chroma_stft(S=stft, sr=sample_rate).T, axis=0)
        result = np.hstack((result, chroma))
    if mel:
        mel = np.mean(librosa.feature.melspectrogram(y=X, sr=sample_rate).T, axis=0)
        result = np.hstack((result, mel))
    return result

# Load the pre-trained model
model = joblib.load('/content/drive/MyDrive/model.pkl')

# Define the user interface widgets
file_upload = widgets.FileUpload(
    accept='.wav',
    multiple=False,
    description='Upload WAV file'
)

recognize_button = widgets.Button(
    description='Recognize Emotion',
    disabled=False
)

output_plot = widgets.Output()

# Define the button click event handler
def recognize_emotion(_):
    file_contents = list(file_upload.value.values())[0]['content']
    with sf.SoundFile(io.BytesIO(file_contents)) as sound_file:
        X = sound_file.read(dtype="float32")
        sample_rate = sound_file.samplerate

    feature = extract_features(X, sample_rate, mfcc=True, chroma=True, mel=True)
    # Apply data augmentation to the feature
    augmented_feature = augment(samples=feature, sample_rate=sample_rate)

    # Predict emotion
    emotions = model.predict_proba([augmented_feature])[0]

    # Display the recognized emotions
    with output_plot:
        output_plot.clear_output(wait=True)
        plt.figure(figsize=(10, 4))
        librosa.display.waveshow(X, sr=sample_rate)
        plt.title(f'Waveform - Predicted Emotions')
        plt.xlabel('Time (s)')
        plt.ylabel('Amplitude')
        plt.show()

        # Display emotion probabilities
        plt.figure(figsize=(6, 4))
        plt.bar(model.classes_, emotions)
        plt.title('Emotion Probabilities')
        plt.xlabel('Emotion')
        plt.ylabel('Probability')
        plt.show()

# Attach the click event handler to the button
recognize_button.on_click(recognize_emotion)

# Display the user interface
display(file_upload, recognize_button, output_plot)


In [ ]:
'''!pip install audiomentations ipywidgets matplotlib
import ipywidgets as widgets
from IPython.display import display, Audio
from sklearn.neural_network import MLPClassifier
import librosa
import soundfile as sf
import numpy as np
import io
import joblib
from audiomentations import Compose, AddGaussianNoise, TimeStretch, PitchShift
import matplotlib.pyplot as plt
import librosa.display

# Data augmentation using audiomentations
augment = Compose([
    AddGaussianNoise(min_amplitude=0.001, max_amplitude=0.015, p=0.5),
    TimeStretch(min_rate=0.8, max_rate=1.2, p=0.5),
    PitchShift(min_semitones=-4, max_semitones=4, p=0.5),
])

# Extract features function (with mel, mfcc, and chroma)
def extract_features(X, sample_rate, mfcc, chroma, mel):
    if chroma:
        stft = np.abs(librosa.stft(X))
    result = np.array([])
    if mfcc:
        mfccs = np.mean(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=40).T, axis=0)
        result = np.hstack((result, mfccs))
    if chroma:
        chroma = np.mean(librosa.feature.chroma_stft(S=stft, sr=sample_rate).T, axis=0)
        result = np.hstack((result, chroma))
    if mel:
        mel = np.mean(librosa.feature.melspectrogram(y=X, sr=sample_rate).T, axis=0)
        result = np.hstack((result, mel))
    return result

# Load the pre-trained model
model = joblib.load('/content/drive/MyDrive/model.pkl')

# Define the user interface widgets
file_upload = widgets.FileUpload(
    accept='.wav',
    multiple=False,
    description='Upload WAV file'
)

recognize_button = widgets.Button(
    description='Recognize Emotion',
    disabled=False
)

output_plot = widgets.Output()

# Define the button click event handler
def recognize_emotion(_):
    file_contents = list(file_upload.value.values())[0]['content']
    with sf.SoundFile(io.BytesIO(file_contents)) as sound_file:
        X = sound_file.read(dtype="float32")
        sample_rate = sound_file.samplerate

    feature = extract_features(X, sample_rate, mfcc=True, chroma=True, mel=True)
    # Apply data augmentation to the feature
    augmented_feature = augment(samples=feature, sample_rate=sample_rate)

    # Predict emotion
    emotions = model.predict_proba([augmented_feature])[0]
    classes = model.classes_

    # Display the recognized emotions
    with output_plot:
        output_plot.clear_output(wait=True)
        plt.figure(figsize=(10, 4))
        librosa.display.waveshow(X, sr=sample_rate)
        plt.title(f'Waveform - Predicted Emotions')
        plt.xlabel('Time (s)')
        plt.ylabel('Amplitude')
        plt.show()

        # Display emotion probabilities
        plt.figure(figsize=(6, 4))
        plt.bar(classes, emotions)
        plt.title('Emotion Probabilities')
        plt.xlabel('Emotion')
        plt.ylabel('Probability')
        plt.show()

# Attach the click event handler to the button
recognize_button.on_click(recognize_emotion)

# Display the user interface
display(file_upload, recognize_button, output_plot)'''
